# Análise de Sentimentos

**Objetivos**
 - Importar dados de comentários para treinar a máquina a prever e avaliar dados futuros.
 - Fazer o pré-processamento de texto adequadamente para análise de sentimento
 - Treinar o modelo vetorizado usando TF-ID com N-Gramas
 - Avaliar métricas de acurácia, precisão, recal e F1 score.
 - Aplicar o aprendizado para prever novos dados

## Importando bibliotecas
Para este projeto, irei utilizar um grande conjunto de bibliotecas:
- **Manipulação de dados:**
  - `pandas` para manipular meus dados e preparar o DataFrame para PLN
      
**Pré-Processamento de texto:**
  - `re` para remoção de diversos caracteres
  - `spacy` para lematização de palavras da lingua portuguesa
  - `nltk` para remoção de stopwords, tokenização e lematização
  - `unidecode` para preservar acentuações da lingua portuguesa
      
**Vetorização e treinamento:**
 - Várias importações de pacotes do `sklearn` para diversos fins de treinamento e vetorização.
 
**Dependências:**
 - Serão baixadas algumas dependências para serem aplicadas no pré-processamento     

In [3]:
import pandas as pd

import re
import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from unidecode import unidecode

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Thiago\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Thiago\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Thiago\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\Thiago\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


True

In [4]:
# Preparando uma função para o pré-processamento de textos em português
nlp = spacy.load('pt_core_news_sm')

def pre_processamento_texto(texto, lemmatize=False):
    texto = re.sub(r'<.*?>', '', texto) # Remove tags HTML
    texto = unidecode(texto) 
    texto = re.sub(r'[^a-zA-Z0-9\s]', '', texto) # Remove caracteres especiais
    texto = re.sub(r'http\S+', '', texto) # Remove URLs
    texto = re.sub(r'\s+', ' ', texto) # Remove espaços em branco extras
    texto = texto.strip() # Remove espaços em branco no início e no final
    texto = texto.lower() # Converte para minúsculas
    texto = re.sub(r'\d+', '', texto) # Remove números
    tokens = word_tokenize(texto) # converte o texto para tokens de palavras
    stop_words = set(stopwords.words('portuguese'))
    tokens = [token for token in tokens if token not in stop_words]
    
    if lemmatize:
        tokens = [token.lemma_ for token in nlp(' '.join(tokens))]

    return " ".join(tokens)

## Preparando dados
Os dados que serão utilizados para treinamento foram gerados por IA. Mas pretendo utilizar um conjunto de dados maiores para melhores resultados

In [5]:
data = pd.read_csv('Comentarios.csv')
df = pd.DataFrame(data)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   positivo  50 non-null     object
 1   negativo  50 non-null     object
 2   neutro    50 non-null     object
dtypes: object(3)
memory usage: 1.3+ KB


In [6]:
df.head()

,positivo,negativo,neutro
0,Este livro foi uma leitura absolutamente incrí...,Este filme foi um desastre total. A atuação er...,"O filme tinha uma história interessante, mas a..."
1,O restaurante tinha um ambiente agradável e a ...,A comida estava horrível e o serviço foi péssi...,"A comida estava ok, nada de muito especial, ma..."
2,Adorei a apresentação! Os artistas foram talen...,Fiquei extremamente decepcionado com o produto...,A apresentação teve seus momentos bons e ruins...
3,Recebi meu pedido e fiquei muito satisfeito co...,"A loja estava suja e desorganizada, e os funci...","O produto era exatamente como estava descrito,..."
4,"A equipe foi muito atenciosa e prestativa, me ...",A apresentação foi monótona e cansativa. O pal...,"A equipe parecia eficiente, mas também não mui..."


In [7]:
# Criando listas para o novo DataFrame
textos = []
sentimentos = []

# Iterando sobre os dados originais para criar as novas listas
for sentimento, lista_textos in data.items():
    for texto in lista_textos:
        textos.append(texto)
        sentimentos.append(sentimento)

# Criando o novo DataFrame
df_pln = pd.DataFrame({'textos': textos, 'sentimentos': sentimentos})

In [8]:
df_pln.head()

,textos,sentimentos
0,Este livro foi uma leitura absolutamente incrí...,positivo
1,O restaurante tinha um ambiente agradável e a ...,positivo
2,Adorei a apresentação! Os artistas foram talen...,positivo
3,Recebi meu pedido e fiquei muito satisfeito co...,positivo
4,"A equipe foi muito atenciosa e prestativa, me ...",positivo


## Pré-processamento
Irei criar uma nova coluna para os textos pré-processados. Durante o processamento, eu evitei a lematização pois estava inteferindo na eficácia da máquina e consequentemente piorando as previsões. Foram feita as remoções dos acentos e do "ç", mas preservando as letras.

In [9]:
# Aplicar pré-processamento
df_pln['textos preprocessados'] = df_pln['textos'].apply(pre_processamento_texto)
df_pln.head()

,textos,sentimentos,textos preprocessados
0,Este livro foi uma leitura absolutamente incrí...,positivo,livro leitura absolutamente incrivel historia ...
1,O restaurante tinha um ambiente agradável e a ...,positivo,restaurante ambiente agradavel comida simplesm...
2,Adorei a apresentação! Os artistas foram talen...,positivo,adorei apresentacao artistas talentosos show d...
3,Recebi meu pedido e fiquei muito satisfeito co...,positivo,recebi pedido fiquei satisfeito qualidade prod...
4,"A equipe foi muito atenciosa e prestativa, me ...",positivo,equipe atenciosa prestativa ajudando todas duv...


## Vetorização e Treinamento
Com o TF-IDF para vetorizar, utilizei o N-Gramas em `1,2` para preservar a sintaxe das frases.
Por alguma razão, a regressão logística obteve melhores resultados do que a multinomialNB.

In [27]:
#------------------
# Vetorização TF-IDF
#------------------
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X = vectorizer.fit_transform(df_pln['textos preprocessados'])
y = df_pln['sentimentos']

#------------------
# Divisão dos dados
#------------------
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#------------------
# Treinamento do modelo (Regressão Logística)
#------------------
model = LogisticRegression()
model.fit(X_train, y_train)

#------------------
# Avaliação do modelo
#------------------
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Acurácia: {accuracy:.2f}")
print(f"Precisão: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-score: {f1:.2f}")

Acurácia: 0.77
Precisão: 0.78
Recall: 0.77
F1-score: 0.76


## Aplicando o Modelo em novos dados

In [23]:
#------------------
# Teste com novos dados
#------------------
novos_coments = [
    "Essa pizza é a coisa mais divina que eu experimentei.",
    "Foi um dos melhores filmes que eu já vi!",
    "Meu Deus, que comida horrível!! Como é que alguém come isso?",
    "O serviço foi razoável, nada de extraordinário, mas também não foi ruim.",
    "Este filme é tão ruim que chega a ser engraçado, mas não no bom sentido.",
    "Estou simplesmente apaixonado por este produto! A qualidade, o design e o atendimento superaram todas as minhas expectativas.",
    "A reunião foi adiada, mas não sei se isso é bom ou ruim."
]
novos_coments_preprocessados = [pre_processamento_texto(review) for review in new_reviews]
novos_coments_vetorizados = vectorizer.transform(novos_coments_preprocessados)
novos_coments_previsao = model.predict(novos_coments_vetorizados)

for review, prediction in zip(novos_coments, novos_coments_previsao):
    print(f"Comentário: {review} - Previsão: {prediction}\n")

Comentário: Essa pizza é a coisa mais divina que eu experimentei. - Previsão: positivo

Comentário: Foi um dos melhores filmes que eu já vi! - Previsão: positivo

Comentário: Meu Deus, que comida horrível!! Como é que alguém come isso? - Previsão: negativo

Comentário: O serviço foi razoável, nada de extraordinário, mas também não foi ruim. - Previsão: neutro

Comentário: Este filme é tão ruim que chega a ser engraçado, mas não no bom sentido. - Previsão: negativo

Comentário: Estou simplesmente apaixonado por este produto! A qualidade, o design e o atendimento superaram todas as minhas expectativas. - Previsão: positivo

Comentário: A reunião foi adiada, mas não sei se isso é bom ou ruim. - Previsão: neutro

